In [1]:
# Temporal Aatching model between Users and Locations
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import calendar
from datetime import datetime
from datetime import timedelta
from scipy import sparse
from scipy.stats import norm
from numpy.random import *
from scipy import optimize

In [2]:
# 連続した日付を取得する関数
def daterange(_start, _end):
    for n in range((_end - _start).days):
        yield _start + timedelta(n)
        
# 切断ポアソン分布を生成する関数
def rtpois(mu, a, b, n):
    FA = scipy.stats.poisson.cdf(a, mu)
    FB = scipy.stats.poisson.cdf(b, mu)
    return np.array(scipy.stats.poisson.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu), dtype="int")

# 多項分布の乱数を生成する関数
def rmnom(pr, n, k, pattern):
    if pattern==1:
        z_id = np.array(np.argmax(np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis], axis=1), dtype="int")
        Z = np.diag(np.repeat(1, k))[z_id, ]
        return z_id, Z
    z_id = np.array(np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1), dtype="int")
    return z_id

In [3]:
# シミュレーションデータを生成
# 日付データを作成
# 対象期間の日付と曜日情報を出力
start = datetime.strptime('2014-01-01', '%Y-%m-%d')
end   = datetime.strptime('2019-09-30', '%Y-%m-%d')
get_date = np.array([])
day_of_week = np.array([], dtype="int")
for day in daterange(start, end):
    get_date = np.append(get_date, day)
    day_of_week = np.append(day_of_week, day.weekday())
    
#日付を定義
index_week = np.array(np.where(day_of_week < 5)[0], dtype="int")
Timestamp = pd.Series(get_date).iloc[index_week]
date = pd.DataFrame({"date": Timestamp.astype("U")})
date_df = pd.merge(date, pd.DataFrame({"date": np.unique(date), "date_id": np.arange(date.shape[0])}), on="date", how="inner")
year = pd.DataFrame({"year": Timestamp.astype("U").str[:4]})
unique_year = np.unique(year)
year_df = pd.merge(year, pd.DataFrame({"year": unique_year, "year_id": np.arange(unique_year.shape[0])}), on="year", how="inner")
month = pd.DataFrame({"month": Timestamp.astype("U").str[5:7]})
unique_month = np.unique(month)
month_df = pd.merge(month, pd.DataFrame({"month": unique_month, "month_id": np.arange(unique_month.shape[0])}), on="month", how="inner")

#週を定義
new_week = day_of_week[index_week]
index = np.array(np.where((new_week-4)==0)[0], dtype="int")
week_n = index.shape[0]
week = np.repeat(0, new_week.shape[0])
for i in range(week_n):
    if i==0:
        get = np.arange(0, index[i]+1)
        week[get] = np.repeat(i, get.shape[0])
    else:
        get = np.arange(index[i-1]+1, index[i]+1)
        week[get] = np.repeat(i, get.shape[0])
        
#データフレームを作成
calendar_df = pd.concat((date_df, year_df, month_df, pd.DataFrame({"week_id": week})), axis=1)
date = np.array(calendar_df["date_id"], dtype="int")
year = np.array(calendar_df["year_id"], dtype="int")
month = np.array(calendar_df["month_id"], dtype="int")
week = np.array(calendar_df["week_id"], dtype="int")

In [4]:
# データの定義
k = 10
hh = 3000
place = 2500
category = 50
area = 25
w = 2
Lambda = np.random.gamma(50.0, 1/0.25, hh)
pt1 = np.random.poisson(Lambda, hh)
hhpt = np.sum(pt1)
k_vec = np.repeat(1.0, k)

In [5]:
# idとインデックスを定義
# idの定義
d_id1 = np.repeat(np.arange(hh), pt1)
pt_id1 = np.array(list(itertools.chain(*[np.array(range(pt1[i]), dtype="int") for i in range(hh)])))

#インデックスを定義
d_list1 = [i for i in range(hh)]
for i in range(hh):
    d_list1[i] = np.array(np.where(d_id1==i)[0], dtype="int")

In [6]:
# placeを選択
#トピックを生成
topic = 25 
theta_topic = np.random.dirichlet(np.repeat(0.2, topic), hh)
phi_place = np.random.dirichlet(np.repeat(0.1, place), topic)
z = np.array(rmnom(theta_topic[d_id1, ], hhpt, topic, 0), dtype="int16")

# 多項分布からplaceを生成
place_id1 = np.repeat(0, hhpt)
for i in range(hh):
    index = d_list1[i]
    place_id1[index] = rmnom(phi_place[z[index], ], pt1[i], place, 0)
    
# インデックスを定義
place_list1 = [i for i in range(place)]
place_n1 = np.repeat(0, place)
for i in range(place):
    place_list1[i] = np.array(np.where(place_id1==i)[0], dtype="int")
    place_n1[i] = place_list1[i].shape[0]

In [7]:
# areaとcategoryを選択
# トピックを生成
theta_place = (phi_place.T) / np.sum(phi_place.T, axis=1)[:, np.newaxis]
phi_area = np.random.dirichlet(np.repeat(0.1, area), topic)
phi_category = np.random.dirichlet(np.repeat(0.1, category), topic)
s = rmnom(theta_place, place, topic, 0)

# 多項分布からareaとcategoryを生成
area_id1 = rmnom(phi_area[s, ], place, area, 0)[place_id1]
category_id1 = rmnom(phi_category[s, ], place, category, 0)[place_id1]

# インデックスを定義
area_list1 = [i for i in range(place)]
category_list1 = [i for i in range(category)]
area_n1 = np.repeat(0, area)
category_n1 = np.repeat(0, category)
for i in range(area):
    area_list1[i] = np.array(np.where(area_id1==i)[0], dtype="int")
    area_n1[i] = area_list1[i].shape[0]
for i in range(category):
    category_list1[i] = np.array(np.where(category_id1==i)[0], dtype="int")
    category_n1[i] = category_list1[i].shape[0]

In [8]:
# 日付と時間を割り当て
# トピックを生成
weekday = 2
hours = 24
phi_weekday = np.random.beta(3.0, 5.0, place)
phi_hours = np.random.dirichlet(np.repeat(0.05, hours), place)
z = np.array(rmnom(theta_topic[d_id1, ], hhpt, topic, 0), dtype="int16")

# 多項分布から日付と時間を生成
day_id1 = np.repeat(0, hhpt)
hours_id1 = np.repeat(0, hhpt)
day_list1 = [i for i in range(w)]
for i in range(hh):
    index = d_list1[i]
    day_id1[index] = np.random.binomial(1, phi_weekday[z[index]], pt1[i])
    hours_id1[index] = rmnom(phi_hours[z[index], ], pt1[i], hours, 0)
for i in range(w):
    day_list1[i] = np.array(np.where(day_id1==i)[0], dtype="int")
wd_index = day_list1[0]
we_index = day_list1[1]

In [9]:
# 時間割当配分を定義
# areaの時間割当
area_wd = np.zeros((area, hours))
area_we = np.zeros((area, hours))
for i in range(area):
    index = area_list1[i]
    freq_wd = np.sum(day_id1[index][:, np.newaxis] * np.diag(np.repeat(1, hours))[hours_id1[index], ], axis=0)
    freq_we = np.sum((1 - day_id1[index])[:, np.newaxis] * np.diag(np.repeat(1, hours))[hours_id1[index], ], axis=0)
    area_wd[i, ] = freq_wd / np.sum(freq_wd)
    area_we[i, ] = freq_we / np.sum(freq_we)
area_rate = np.zeros((area, hours, w))
area_rate[:, :, 0] = area_wd; area_rate[:, :, 1] = area_we

# categoryの時間割当
category_wd = np.zeros((category, hours))
category_we = np.zeros((category, hours))
for i in range(category):
    index = category_list1[i]
    freq_wd = np.sum(day_id1[index][:, np.newaxis] * np.diag(np.repeat(1, hours))[hours_id1[index], ], axis=0)
    freq_we = np.sum((1 - day_id1[index])[:, np.newaxis] * np.diag(np.repeat(1, hours))[hours_id1[index], ], axis=0)
    category_wd[i, ] = freq_wd / np.sum(freq_wd)
    category_we[i, ] = freq_we / np.sum(freq_we)
category_rate = np.zeros((category, hours, w))
category_rate[:, :, 0] = category_wd; category_rate[:, :, 1] = category_we

In [10]:
# パラメータを生成
# 事前分布の定義
alpha_u = 2*np.random.dirichlet(np.repeat(0.25, k), hh)
alpha_d = np.random.gamma(0.5, 1.5, hh*hours).reshape(hh, hours)
alpha_v = np.random.gamma(1.0, 1.75, place*k).reshape(place, k)

# モデルパラメータを生成
pi = np.random.beta(2.5, 3.0, place)
theta_u = np.zeros((hh, k, w))
theta_d = np.zeros((hh, hours, w))
theta_v = np.zeros((place, k, w))
for i in range(hh):
    for j in range(w):
        theta_u[i, :, j] = np.random.dirichlet(alpha_u[i, ], 1).reshape(-1)
        theta_d[i, :, j] = np.random.gamma(0.5, alpha_d[i, ], hours)
for i in range(place):
    for j in range(w):
        theta_v[i, :, j] = np.random.gamma(0.5, alpha_v[i, ], k)
        
# パラメータの真値をコピー
pit = pi.copy()
alphat_u = alpha_u.copy()
alphat_d = alpha_d.copy()
alphat_v = alpha_v.copy()
thetat_u = theta_u.copy()
thetat_d = theta_d.copy()
thetat_v = theta_v.copy()

In [11]:
# 応答変数を生成
# areaとcategoryの割当トピックを生成
Z = np.random.binomial(1, pi[place_id1], hhpt)
z_vec = Z[place_id1][:, np.newaxis]

# 期待値を定義
mv_weights = z_vec*area_rate[area_id1, :, day_id1] + (1-z_vec)*category_rate[category_id1, :, day_id1]
mv = np.dot(theta_d[d_id1, :, day_id1] * mv_weights, np.repeat(1.0, hours))
uv = np.dot(theta_u[d_id1, :, day_id1] * theta_v[place_id1, :, day_id1], k_vec)
mu = mv + uv

# Bernoulli Poisson linkからbinaryデータを生成
freq = np.random.poisson(mu, hhpt)
x = np.array(freq > 0, dtype="int")
index_x = np.array(np.where(x==1)[0], dtype="int")
N = index_x.shape[0]

In [12]:
# 新しいidとインデックスを定義
# idを定義
d_id2 = d_id1[index_x]
place_id2 = place_id1[index_x]
area_id2 = area_id1[index_x]
category_id2 = category_id1[index_x]
day_id2 = day_id1[index_x]
hours_id2 = hours_id1[index_x]

# インデックスを定義
d_list2 = [i for i in range(hh)]
place_list2 = [i for i in range(place)]
area_list2 = [i for i in range(area)]
category_list2 = [i for i in range(category)]
day_list2 = [i for i in range(w)]
hours_list2 = [i for i in range(hours)]
pt2 = np.repeat(0, hh)
place_n2 = np.repeat(0, hh)
area_n2 = np.repeat(0, hh)
category_n2 = np.repeat(0, hh)

for i in range(hh):
    d_list2[i] = np.array(np.where(d_id2==i)[0], dtype="int")
    pt2[i] = d_list2[i].shape[0]
pt_id2 = np.array(list(itertools.chain(*[np.array(range(pt2[i]), dtype="int") for i in range(hh)])))
for i in range(place):
    place_list2[i] = np.array(np.where(place_id2==i)[0], dtype="int")
    place_n2[i] = place_list2[i].shape[0]
for i in range(area):
    area_list2[i] = np.array(np.where(area_id2==i)[0], dtype="int")
    area_n2[i] = area_list2[i].shape[0]
for i in range(category):
    category_list2[i] = np.array(np.where(category_id2==i)[0], dtype="int")
    category_n2[i] = category_list2[i].shape[0]
for i in range(hours):
    hours_list2[i] = np.array(np.where(hours_id2==i)[0], dtype="int")
for i in range(w):
    day_list2[i] = np.array(np.where(day_id2==i)[0], dtype="int")

In [86]:
# Temporal Matching model between users and locationsのパラメータを推定
# MCMCの設定
R = 1000
keep = 2
burnin = int(500/keep)
iter = 0
disp = 10

# データの定義
day_dt1 = np.diag(np.repeat(1, w))[day_id1, ]
day_dt2 = np.diag(np.repeat(1, w))[day_id2, ]
day_vec = np.repeat(1.0, hours)
freq_y = freq[index_x]

# 事前分布の定義
alpha11 = 1.0
alpha12 = 0.1
alpha21 = 0.05; beta21 = 0.05
alpha22 = 0.01; beta22 = 0.01
alpha31 = 0.05; beta31 = 0.05
alpha32 = 0.01; beta32 = 0.01
s0 = 0.5; v0 = 0.5

In [87]:
# パラメータの真値
# モデルパラメータの真値
pi = pit.copy()
alpha_u = alphat_u.copy()
alpha_d = alphat_d.copy()
alpha_v = alphat_v.copy()
theta_u = thetat_u.copy()
theta_d = thetat_d.copy()
theta_v = thetat_v.copy()

# areaとcategoryの割当トピックを生成
Zi = Z.copy()
z_vec = Zi[:, np.newaxis]

# 期待値を定義
mv_weights = z_vec*area_rate[area_id1, :, day_id1] + (1-z_vec)*category_rate[category_id1, :, day_id1]
mv = np.dot(theta_d[d_id1, :, day_id1] * mv_weights, np.repeat(1.0, hours))
uv = np.dot(theta_u[d_id1, :, day_id1] * theta_v[place_id1, :, day_id1], k_vec)
mu = mv + uv

In [88]:
# パラメータの初期値
# モデルパラメータの初期値
pi = np.random.beta(10.0, 10.0, place)
alpha_u = np.random.dirichlet(np.repeat(0.5, k), hh)
alpha_d = np.random.gamma(1.0, 0.25, hh*hours).reshape(hh, hours)
alpha_v = np.random.gamma(1.0, 0.25, place*k).reshape(place, k)
theta_u = np.zeros((hh, k, w))
theta_d = np.zeros((hh, hours, w))
theta_v = np.zeros((place, k, w))
for i in range(hh):
    for j in range(w):
        theta_u[i, :, j] = np.random.dirichlet(alpha_u[i, ], 1).reshape(-1)
        theta_d[i, :, j] = np.random.gamma(0.5, alpha_d[i, ], hours)
for i in range(place):
    for j in range(w):
        theta_v[i, :, j] = np.random.gamma(0.5, alpha_v[i, ], k)
        
# areaとcategoryの割当トピックを生成
Zi = np.random.binomial(1, pi[place_id1], hhpt)
z_vec = Zi[:, np.newaxis]

# 期待値を定義
mv_weights = z_vec*area_rate[area_id1, :, day_id1] + (1-z_vec)*category_rate[category_id1, :, day_id1]
mv = np.dot(theta_d[d_id1, :, day_id1] * mv_weights, np.repeat(1.0, hours))
uv = np.dot(theta_u[d_id1, :, day_id1] * theta_v[place_id1, :, day_id1], k_vec)
mu = mv + uv

In [89]:
# パラメータの格納用配列
PI = np.zeros((int(R/keep), place))
ALPHA_U = np.zeros((hh, k, int(R/keep)))
ALPHA_D = np.zeros((hh, hours, int(R/keep)))
ALPHA_V = np.zeros((place, k, int(R/keep)))
THETA_U = np.zeros((hh, k, w, int(R/keep)))
THETA_D = np.zeros((hh, hours, w, int(R/keep)))
THETA_V = np.zeros((place, k, w, int(R/keep)))
SEG = np.zeros((int(R/keep), hhpt), dtype="int8")

In [90]:
# 対数尤度の基準値
# 1パラメータモデルの対数尤度
LLst = np.sum(scipy.stats.poisson.logpmf(freq, np.mean(freq)))
print(LLst)

# 真値での対数尤度
z_long = Z[:, np.newaxis]
mv_weights = z_long*area_rate[area_id1, :, day_id1] + (1-z_long)*category_rate[category_id1, :, day_id1]
mv = np.dot(thetat_d[d_id1, :, day_id1] * mv_weights, day_vec)
uv = np.dot(thetat_u[d_id1, :, day_id1] * thetat_v[place_id1, :, day_id1], k_vec)
mut = mv + uv
LLbest = np.sum(scipy.stats.poisson.logpmf(freq, mut))
print(LLbest)

-1059390.0653573389
-743187.9981879352


In [91]:
# ギブスサンプリングでパラメータをサンプリング
for rp in range(R):
    
    # 期待値を定義
    mv_weights = z_vec[index_x] *area_rate[area_id2, :, day_id2] + (1-z_vec[index_x])*category_rate[category_id2, :, day_id2]
    mv_deploy = theta_d[d_id2, :, day_id2] * mv_weights
    mv = np.dot(mv_deploy, day_vec)
    uv_deploy = theta_u[d_id2, :, day_id2] * theta_v[place_id2, :, day_id2]
    uv = np.dot(uv_deploy, k_vec)
    mu = mv + uv

    # ユーザーのトピック分布を更新
    # 補助変数を定義
    Lambda = uv_deploy / mu[:, np.newaxis]
    lambda_y = freq_y[:, np.newaxis] * Lambda

    # ディリクリ分布からパラメータをサンプリング
    for i in range(hh):
        index = d_list2[i]
        wsum = np.dot(day_dt2[index, ].T, lambda_y[index, ])
        wsum_u = wsum + alpha11*alpha_u[i, ] + alpha12
        for j in range(w):
            theta_u[i, :, j] = np.random.dirichlet(wsum_u[j, ] , 1).reshape(-1)
        alpha_u[i, ] = np.random.dirichlet(np.sum(wsum, axis=0) + alpha12, 1).reshape(-1)

    # 期待値を更新
    uv_deploy = theta_u[d_id2, :, day_id2] * theta_v[place_id2, :, day_id2]
    uv = np.dot(uv_deploy, k_vec)
    mu = mv + uv


    # placeの特徴ベクトルを更新
    # 補助変数を定義
    Lambda = uv_deploy / mu[:, np.newaxis]
    lambda_y = freq_y[:, np.newaxis] * Lambda
    lambda_h = theta_u[d_id1, :, day_id1]

    # placeごとにパラメータをサンプリング
    W21 = np.zeros((place, k))
    W22 = np.zeros((place, k))
    for i in range(place):

        # 事後分布のパラメータ
        index1 = place_list1[i]; index2 = place_list2[i]
        get_day1 = day_dt1[index1, ].T
        get_day2 = day_dt2[index2, ].T
        W11 = np.dot(get_day2, lambda_y[index2, ])
        W12 = np.dot(get_day1, lambda_h[index1, ])
        W21[i, ] = np.sum(W11, axis=0) + alpha22
        W22[i, ] = np.sum(W12, axis=0) + beta22

        # ガンマ分布からパラメータをサンプリング
        phi = W11 + alpha21*alpha_v[i, ] + alpha22 
        kappa = W12 + beta21 + beta22
        for j in range(w):
            theta_v[i, :, j] = np.random.gamma(phi[j, ], 1/kappa[j, ])
    alpha_v = np.random.gamma(W21, 1/W22)

    # 期待値を更新
    uv_deploy = theta_u[d_id2, :, day_id2] * theta_v[place_id2, :, day_id2]
    uv = np.dot(uv_deploy, k_vec)
    mu = mv + uv


    # areaとcategoryの重みを更新
    # トピック毎の期待値を定義
    day_deploy = theta_d[d_id1, :, day_id1]
    mv1 = np.dot(day_deploy * area_rate[area_id1, :, day_id1], day_vec)[:, np.newaxis]
    mv2 = np.dot(day_deploy * category_rate[area_id1, :, day_id1], day_vec)[:, np.newaxis]
    uv_long = np.dot(theta_u[d_id1, :, day_id1] * theta_v[place_id1, :, day_id1], k_vec)[:, np.newaxis]

    # 重みの割当確率
    Prior = np.hstack((pi[:, np.newaxis], (1-pi)[:, np.newaxis]))[place_id1, ] 
    Posterior = Prior * scipy.stats.poisson.pmf(freq[:, np.newaxis], uv_long + np.hstack((mv1, mv2)))
    Prob = Posterior[:, 0] / np.sum(Posterior, axis=1)

    # ベルヌーイ分布から重みをサンプリング
    Zi = np.random.binomial(1, Prob, hhpt)
    z_vec = Zi[:, np.newaxis]
    mv_weights = z_vec[index_x] * area_rate[area_id2, :, day_id2] + (1-z_vec[index_x])*category_rate[category_id2, :, day_id2]
    mv_deploy = theta_d[d_id2, :, day_id2] * mv_weights
    mv = np.dot(mv_deploy, day_vec)
    mu = mv + uv

    # ベータ分布から事前分布を更新
    s = np.repeat(0.0, place); v = np.repeat(0.0, place)
    for i in range(place):
        index = place_list1[i]
        s[i] = np.sum(z_vec[index]) + s0
        v[i] = place_n1[i] - np.sum(z_vec[index]) + v0
    pi = np.random.beta(s, v, place)


    # 時間の重み係数を更新
    # 補助変数を更新
    Lambda = mv_deploy / mu[:, np.newaxis]
    lambda_y = freq_y[:, np.newaxis] * Lambda
    lambda_h = z_vec*area_rate[area_id1, :, day_id1] + (1-z_vec)*category_rate[category_id1, :, day_id1]

    # ユーザーごとにパラメータをサンプリング
    W21 = np.zeros((hh, hours))
    W22 = np.zeros((hh, hours))
    for i in range(hh):

        # 事後分布のパラメータ
        index1 = d_list1[i]; index2 = d_list2[i]
        get_day1 = day_dt1[index1, ].T
        get_day2 = day_dt2[index2, ].T
        W11 = np.dot(get_day2, lambda_y[index2, ])
        W12 = np.dot(get_day1, lambda_h[index1, ])
        W21[i, ] = np.sum(W11, axis=0) + alpha32
        W22[i, ] = np.sum(W12, axis=0) + beta32

        # ガンマ分布からパラメータをサンプリング
        phi = W11 + alpha31*alpha_d[i, ] + alpha32 
        kappa = W12 + beta31 + beta32
        for j in range(w):
            theta_d[i, :, j] = np.random.gamma(phi[j, ], 1/kappa[j, ])
    alpha_d = np.random.gamma(W21, 1/W22)


    # サンプリング結果の格納と表示
    # サンプリング結果の格納
    if rp%keep==0:
        mkeep = int(rp/keep)
        PI[mkeep, ] = pi
        ALPHA_U[:, :, mkeep] = alpha_u
        ALPHA_D[:, :, mkeep] = alpha_d
        ALPHA_V[:, :, mkeep] = alpha_v
        THETA_U[:, :, :, mkeep] = theta_u
        THETA_D[:, :, :, mkeep] = theta_d
        THETA_V[:, :, :, mkeep] = theta_v
        SEG[mkeep, ] = Zi

    #対数尤度の更新とサンプリング結果の表示   
    if rp%disp==0:
        # 対数尤度の更新
        mv_weights = z_vec*area_rate[area_id1, :, day_id1] + (1-z_vec)*category_rate[category_id1, :, day_id1]
        mv = np.dot(theta_d[d_id1, :, day_id1] * mv_weights, day_vec)
        uv = np.dot(theta_u[d_id1, :, day_id1] * theta_v[place_id1, :, day_id1], k_vec)
        mu = mv + uv
        LL = np.sum(scipy.stats.poisson.logpmf(freq, mu))
        
        #サンプリング結果の表示
        print(rp)
        print(np.round([LL, LLst, LLbest], 1))

0
[-1104536.3 -1059390.1  -743188. ]
10
[ -914067.1 -1059390.1  -743188. ]
20
[ -897309.9 -1059390.1  -743188. ]
30
[ -885461.6 -1059390.1  -743188. ]
40
[ -874133.7 -1059390.1  -743188. ]
50
[ -864217.1 -1059390.1  -743188. ]
60
[ -854546.  -1059390.1  -743188. ]
70
[ -846066.9 -1059390.1  -743188. ]
80
[ -837649.8 -1059390.1  -743188. ]
90
[ -831426.7 -1059390.1  -743188. ]
100
[ -824220.7 -1059390.1  -743188. ]
110
[ -819166.3 -1059390.1  -743188. ]
120
[ -813684.2 -1059390.1  -743188. ]
130
[ -809579.8 -1059390.1  -743188. ]
140
[ -804638.5 -1059390.1  -743188. ]
150
[ -801045.2 -1059390.1  -743188. ]
160
[ -798076.8 -1059390.1  -743188. ]
170
[ -794578.4 -1059390.1  -743188. ]
180
[ -792425.2 -1059390.1  -743188. ]
190
[ -789984.4 -1059390.1  -743188. ]
200
[ -787782.7 -1059390.1  -743188. ]
210
[ -786233.7 -1059390.1  -743188. ]
220
[ -784883.5 -1059390.1  -743188. ]
230
[ -783155.9 -1059390.1  -743188. ]
240
[ -782139.8 -1059390.1  -743188. ]
250
[ -780753.9 -1059390.1  -743188.